In [3]:
import typing
dataset: typing.List[typing.Tuple[str, str]] = []
with open("assets/text/mandarin_train.txt", "r", encoding='utf-8') as f:
	lines = f.readlines()
	for i in range(0, len(lines), 2):
		line = lines[i].strip().strip('\n')
		if line == "": continue
		phome = lines[i+1].strip().strip('\n')
		dataset.append((line, phome))
print(f"dataset: {len(dataset)}")

dataset: 1205


In [4]:
dataset[:5]

[('躺在急救担架上的男子双目紧闭，头发散发出一股烧焦的味道。',
  'tang3 zai4 ji2 jiu4 dan1 jia4 shang4 de5 nan2 zi3 shuang1 mu4 jin3 bi4 tou2 fa4 san4 fa1 chu1 yi4 gu3 shao1 jiao1 de5 wei4 dao4'),
 ('工业园区是承接产业转移、加速产业集聚、培育产业集群的主要载体。',
  'gong1 ye4 yuan2 qu1 shi4 cheng2 jie1 chan3 ye4 zhuan3 yi2 jia1 su4 chan3 ye4 ji2 ju4 pei2 yu4 chan3 ye4 ji2 qun2 de5 zhu3 yao4 zai4 ti3'),
 ('那一刻，我才真正的懂你，就像懂我现在的自己。',
  'na4 yi2 ke4 wo3 cai2 zhen1 zheng4 de5 dong2 ni3 jiu4 xiang4 dong2 wo3 xian4 zai4 de5 zi4 ji3'),
 ('由于列车长时间停靠，车厢内的空气越来越“闷”。',
  'you2 yu2 lie4 che1 zhang3 shi2 jian1 ting2 kao4 che1 xiang1 nei4 de5 kong1 qi4 yue4 lai2 yue4 men1'),
 ('但如果按车队规模，一嗨数千辆车的量级绝对算不上最大。',
  'dan4 ru2 guo3 an4 che1 dui4 gui1 mo2 yi4 hai1 shu4 qian1 liang4 che1 de5 liang4 ji2 jue2 dui4 suan4 bu2 shang4 zui4 da4')]

In [2]:
import jieba, pypinyin

In [43]:
def calpinyin_str(text: str):
	cuts = list(jieba.cut(text))
	pinyin = []
	for cut in cuts:
		if cut in ["，", "。", "\"", "”", "“", "、","？","！", "；","："]: continue
		py = pypinyin.lazy_pinyin(cut, style=pypinyin.Style.TONE3, tone_sandhi=True)
		for x in py:
			pinyin.append(x)
	return ' '.join(pinyin).strip()

In [44]:
need_to_fixed: typing.List[typing.Tuple[int, str]] = []
auto_fixed: typing.List[typing.Tuple[int, str]] = []
for i, (text, phome) in enumerate(dataset):
	pinyin_str = calpinyin_str(text)
	if pinyin_str != phome:
		source_pinyin = dataset[i][1].split(' ')
		pinyin = pinyin_str.split(' ')
		can_auto_fix = True
		target_pinyin = []
		if len(source_pinyin) == len(pinyin):
			for k in range(len(source_pinyin)):
				if source_pinyin[k]== pinyin[k]:
					target_pinyin.append(source_pinyin[k])
				elif source_pinyin[k].startswith(pinyin[k]) and source_pinyin[k][len(pinyin[k]):] in ["1", "2", "3", "4", "5"]:
					target_pinyin.append(pinyin[k])
				else:
					can_auto_fix = False
					break
		else:
			can_auto_fix = False
			
		if can_auto_fix:
			auto_fixed.append((i, ' '.join(target_pinyin)))
		else:
			need_to_fixed.append((i, pinyin_str))
print(f"need to fixed: {len(need_to_fixed)}")
print(f"auto fixed: {len(auto_fixed)}")

need to fixed: 970
auto fixed: 172


In [37]:
showindex = 0
showlist = need_to_fixed
print(dataset[showlist[showindex][0]][0])
print("Pinyin:")
print(showlist[showindex][1])
print("Source:")
print(dataset[showlist[showindex][0]][1])

躺在急救担架上的男子双目紧闭，头发散发出一股烧焦的味道。
Pinyin:
tang3 zai4 ji2 jiu4 dan1 jia4 shang4 de nan2 zi shuang1 mu4 jin3 bi4 tou2 fa4 san4 fa4 chu1 yi1 gu3 shao1 jiao1 de wei4 dao4
Source:
tang3 zai4 ji2 jiu4 dan1 jia4 shang4 de5 nan2 zi3 shuang1 mu4 jin3 bi4 tou2 fa4 san4 fa1 chu1 yi4 gu3 shao1 jiao1 de5 wei4 dao4


In [38]:
import json

In [47]:
data = []
for (line_number, pinyin_str) in need_to_fixed:
	source = dataset[line_number][1].split(' ')
	pinyin = pinyin_str.split(' ')
	if len(source) == len(pinyin):
		for i in range(len(source)):
			if source[i]!= pinyin[i]:
				if source[i].startswith(pinyin[i]) and source[i][len(pinyin[i]):] in ["1", "2", "3", "4", "5"]:
					source[i] = pinyin[i]
				elif pinyin[i].startswith(source[i]) and pinyin[i][len(source[i]):] in ["1", "2", "3", "4", "5"]:
					pinyin[i] = source[i]
				else:
					source[i] = f"[{source[i]}]"
					pinyin[i] = f"[{pinyin[i]}]"
	data.append({
		"line": line_number,
		"text": dataset[line_number][0],
		"pinyin": ' '.join(pinyin),
		"source": ' '.join(source),
		"target": ' '.join(pinyin)
	})
with open('need-to-fix.json', 'w', encoding='utf-8') as f:
	f.write(json.dumps(data, indent=4, ensure_ascii=False))


In [48]:
with open("need-to-fix.json", "r", encoding='utf-8') as f:
	jsondata = json.load(f)

In [51]:
jsondata.sort(key=lambda x: x["line"])

In [54]:
print(jsondata[10])
print(need_to_fixed[10])

{'line': 13, 'text': '高油价将对国际关系的互动，产生某种微妙或带有务实倾向的影响。', 'pinyin': 'gao1 you2 jia4 jiang1 dui4 guo2 ji4 guan1 [xi4] de hu4 dong4 chan3 sheng1 [mou3] zhong3 wei1 miao4 huo4 dai4 you3 wu4 shi2 qing1 xiang4 de [ying3] xiang3', 'source': 'gao1 you2 jia4 jiang1 dui4 guo2 ji4 guan1 [xi5] de hu4 dong4 chan3 sheng1 [mou2] zhong3 wei1 miao4 huo4 dai4 you3 wu4 shi2 qing1 xiang4 de [ying2] xiang3', 'target': 'gao1 you2 jia4 jiang1 dui4 guo2 ji4 guan1 [xi4] de hu4 dong4 chan3 sheng1 [mou2] zhong3 wei1 miao4 huo4 dai4 you3 wu4 shi2 qing1 xiang4 de [ying2] xiang3'}
(13, 'gao1 you2 jia4 jiang1 dui4 guo2 ji4 guan1 xi4 de hu4 dong4 chan3 sheng1 mou3 zhong3 wei1 miao4 huo4 dai4 you3 wu4 shi2 qing1 xiang4 de ying3 xiang3')


In [65]:
new_need_tofixed = need_to_fixed.copy()
for i in range(len(jsondata)):
	if jsondata[i]["line"] <= 174:
		target = jsondata[i]["target"]
	else:
		target = ' '.join([ x[:-1] if x.endswith('5') else x for x in jsondata[i]["source"].split(' ') ])
	target = target.replace('[','').replace(']','')
	new_need_tofixed[i] = (new_need_tofixed[i][0], target)

In [66]:
new_need_tofixed[59]

(69,
 'zuo2 tian1 de mi2 wu4 he2 reng2 ran2 tuan1 ji2 bu2 guo4 che1 liang4 yi3 jing1 neng2 gou4 fei1 kuai4 de tong1 guo4')

In [67]:
corrected = dataset.copy()
for (line_number, phome) in new_need_tofixed:
	corrected[line_number] = (corrected[line_number][0], phome)
for (line_number, phome) in auto_fixed:
	corrected[line_number] = (corrected[line_number][0], phome)
corrected[:5]

[('躺在急救担架上的男子双目紧闭，头发散发出一股烧焦的味道。',
  'tang3 zai4 ji2 jiu4 dan1 jia4 shang4 de nan2 zi shuang1 mu4 jin3 bi4 tou2 fa4 san4 fa1 chu1 yi4 gu3 shao1 jiao1 de wei4 dao4'),
 ('工业园区是承接产业转移、加速产业集聚、培育产业集群的主要载体。',
  'gong1 ye4 yuan2 qu1 shi4 cheng2 jie1 chan3 ye4 zhuan3 yi2 jia1 su4 chan3 ye4 ji2 ju4 pei2 yu4 chan3 ye4 ji2 qun2 de zhu3 yao4 zai4 ti3'),
 ('那一刻，我才真正的懂你，就像懂我现在的自己。',
  'na4 yi2 ke4 wo3 cai2 zhen1 zheng4 de dong2 ni3 jiu4 xiang4 dong3 wo3 xian4 zai4 de zi4 ji3'),
 ('由于列车长时间停靠，车厢内的空气越来越“闷”。',
  'you2 yu2 lie4 che1 zhang3 shi2 jian1 ting2 kao4 che1 xiang1 nei4 de kong1 qi4 yue4 lai2 yue4 men1'),
 ('但如果按车队规模，一嗨数千辆车的量级绝对算不上最大。',
  'dan4 ru2 guo3 an4 che1 dui4 gui1 mo2 yi1 hai1 shu4 qian1 liang4 che1 de liang4 ji2 jue2 dui4 suan4 bu2 shang4 zui4 da4')]

In [68]:
with open("assets/text/mandarin_train_partial_clean.txt", "w", encoding='utf-8') as f:
	for (line, pinyin) in corrected:
		f.write(line + "\n" + pinyin + "\n")